In [ ]:
#Imports
from scipy.optimize import minimize

In [ ]:
Cinlet = 20 #mole/L
Vtot = 20 #m3
q = 0.03 #m3/s
n = 2.4
k = 0.0062 #(mole/l)**-n/s
V2 = 10
V1 = Vtot - V2

In [ ]:
def obj_func(Cin, Coutlet):
    return -Coutlet/Cin
"""
descision_variables = [V1, V2]

constraints = ({'type': 'eq', 'fun': lambda x: x[0] + x[1] - Vtot})

sol = minimize(obj_func, descision_variables, constraints=constraints)
"""

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize, fsolve
from scipy.integrate import solve_ivp
def cstrobjective(V,params):
    # decision varaiables, V(1:N-1)
    params['V'] = np.hstack(( V, params['Vtot']-np.sum(V) ))
    cout = cstrsolve(params)
    # maximze conversion
    conv = 1-cout/params['c0']
    # objective: minimize 1-conversion
    obj = 1-conv
    return obj
def cstrsolve(params):
    # solve for steady-state
    cinit = 0.5*params['c0']*np.ones(params['V'].shape)
    #print(f"cinit: {cinit}")
    cstat = fsolve(lambda c: cstrmodel(0,c,params), cinit)
    print(f"cstat: {cstat}")
    # outlet conc.
    cout = cstat[-1]
    return cout
def cstrmodel(t,c,params):
    # reaction rate
    r = params['k']*c**params['n']
    #print(f"r: {r}")
    # material balances
    cm1 = np.hstack(( params['c0'],c[:-1] ))
    #print(f"cm1: {cm1}")
    dcdt = params['q']/params['V']*(cm1 - c) - r    
    #print(f"dcdt: {dcdt}")
    return dcdt
params = {}
#number of cstr
params['N'] = 2
# total cstr series volume
params['Vtot']=20
# model paramsameters
params['c0'] = 20
params['n'] = 2.4
params['k'] = 0.0062
params['q'] = 0.03
Vguess = 10

In [ ]:
#Unconstraint optimizaiton
# optimization using Nelder-Mead
print('==== Nelder-Mead ====')
optim1 = {'disp':True}
res1 = minimize(lambda V: cstrobjective(V,params), Vguess, method='Nelder-Mead', options=optim1)
Vf1 = res1['x']
print(f'Nelder-Mead - x: {res1["x"]}')

In [ ]:
# optimization using BFGS
print('==== BFGS ====')
optim2 = {'disp':True}
res2 = minimize(lambda V: cstrobjective(V,params), Vguess, method='BFGS', options=optim2)
Vf2 = res2['x']
print(f'BFGS - x: {res2["x"]}')